<a href="https://colab.research.google.com/github/1028Luo/LLAMA_Light_Workout/blob/main/compress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q transformers

In [2]:
# Log in to Hugging Face
from google.colab import userdata
from huggingface_hub import login
my_hugging_face_token = userdata.get('huggingface_token')
login(token=my_hugging_face_token)

# Pruning

In [5]:
import torch
import torch.nn.utils.prune as prune
import torch.nn as nn
from transformers import AutoModelForCausalLM

# Load model (you can change this to any causal LLM)
model_name = "Jiexing1028/llama-3-8b-workout"  # Or your quantized/fine-tuned model
model = AutoModelForCausalLM.from_pretrained(model_name)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


model.eval()
device = torch.device("cpu")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# === Prune Linear Layers ===
def apply_magnitude_pruning(model, amount=0.3):
    print(f"Pruning {amount*100:.0f}% of weights in Linear layers...")
    for name, module in model.named_modules():
        print(name)
        # check if a variable is an instance of the second arguement class
        if isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
    return model

# Optional: Make pruning permanent (remove the reparam hooks)
def remove_pruning_wrappers(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            try:
                prune.remove(module, 'weight')
            except ValueError:
                pass  # Already removed or not pruned
    return model

# Apply pruning
prune_amount = 0.3  # 30% sparsity
model = apply_magnitude_pruning(model, amount=prune_amount)

# Optional: Remove the pruning hooks and save pruned model
# model = remove_pruning_wrappers(model)
# model.save_pretrained("pruned-model")

print("Model saved to ./pruned-model")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Total parameters: 8,037,076,992
Trainable parameters: 0
Pruning 30% of weights in Linear layers...

model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.q_proj.base_layer
model.layers.0.self_attn.q_proj.lora_dropout
model.layers.0.self_attn.q_proj.lora_dropout.default
model.layers.0.self_attn.q_proj.lora_A
model.layers.0.self_attn.q_proj.lora_A.default
model.layers.0.self_attn.q_proj.lora_B
model.layers.0.self_attn.q_proj.lora_B.default
model.layers.0.self_attn.q_proj.lora_embedding_A
model.layers.0.self_attn.q_proj.lora_embedding_B
model.layers.0.self_attn.q_proj.lora_magnitude_vector
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.v_proj.base_layer
model.layers.0.self_attn.v_proj.lora_dropout
model.layers.0.self_attn.v_proj.lora_dropout.default
model.layers.0.self_attn.v_proj.lora_A
model.layers.0.self_attn.v_proj.lora_A.default
model.layers.0.self_attn.v_proj.lo

KeyboardInterrupt: 

# Quantization

# export to ONNX